In [2]:
'''

=================================================
Milestone 3

Nama  : Mirza Rendra Sjarief
Batch : CODA-001-RMT

Program ini dibuat untuk melakukan data explorasi sederhana menggunakan Pandas DataFrame 
yang dilanjutkan dengan transform dataset menggunakan PySpark. Adapun dataset yang dipakai adalah 
dataset tentang Us Store Sales selama periode tahun 2010 - 2011

=================================================

'''

'\n\n=================================================\nMilestone 3\n\nNama  : Mirza Rendra Sjarief\nBatch : CODA-001-RMT\n\nProgram ini dibuat untuk melakukan data explorasi sederhana menggunakan Pandas DataFrame \nyang dilanjutkan dengan transform dataset menggunakan PySpark. Adapun dataset yang dipakai adalah \ndataset tentang Us Store Sales selama periode tahun 2010 - 2011\n\n=================================================\n\n'

In [3]:
# import necessary libraries 
import pandas as pd
import numpy as np
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, concat, lit, when, to_date, to_timestamp, upper, lower, date_format,concat_ws,hash,abs,monotonically_increasing_id

### Pandas DataFrame Exploration

- Sebelum melakukan transformasi data dengan Pyspark. dataset ini akan melalui proses pengecekan atau eksplorasi data dengan menggunakan Library Pandas DataFrame, karena lebih flexibel dan memiliki fitur yang sedikit lebih lengkap dibandingkan dengan pyspark dalam hal explorasi data.

In [4]:
#Loading dataset to Pandas DataFrame
df = pd.read_csv('/Users/mac/Documents/Hacktiv8/M3/p2-coda001-rmt-m3-mirzasjarief/us_store_sales.csv', low_memory=False)
pd.set_option("display.max_columns", None)

df.head()

,Area Code,State,Market,Market Size,Profit,Margin,Sales,COGS,Total Expenses,Marketing,Inventory,Budget Profit,Budget COGS,Budget Margin,Budget Sales,ProductId,Date,Product Type,Product,Type
0,203,Connecticut,East,Small Market,107.0,176.0,292.0,116.0,69.0,38.0,962.0,110.0,110.0,160.0,270.0,2,04/01/10 00:00:00,Coffee,Columbian,Regular
1,203,Connecticut,East,Small Market,75.0,135.0,225.0,90.0,60.0,29.0,1148.0,90.0,80.0,130.0,210.0,2,07/01/10 00:00:00,Coffee,Columbian,Regular
2,203,Connecticut,East,Small Market,122.0,195.0,325.0,130.0,73.0,42.0,1134.0,130.0,110.0,180.0,290.0,2,11/01/10 00:00:00,Coffee,Columbian,Regular
3,203,Connecticut,East,Small Market,105.0,174.0,289.0,115.0,69.0,37.0,1166.0,110.0,100.0,160.0,260.0,2,12/01/10 00:00:00,Coffee,Columbian,Regular
4,203,Connecticut,East,Small Market,104.0,135.0,223.0,90.0,56.0,29.0,1148.0,90.0,80.0,130.0,210.0,2,07/01/11 00:00:00,Coffee,Columbian,Regular


- Berikut adalah background dataset yang diguanakan :

- Source    : kaggle.com
- URL       : https://www.kaggle.com/datasets/dsfelix/us-stores-sales
- Author    : Ds Felix
- Licence   : CC0: Public Domain

In [5]:
# Computing summarizing diplicates
df.duplicated().sum()

0

- Langkah pertama adalah melakukan pengecekan duplikat data, hasil menunjukan tidak adanya data duplikat pada dataset ini sehingga bisa dilakukan tahap pengecekan selanjutnya.

In [15]:
# Computing describe function
df[['Area Code','Inventory','ProductId','Sales','COGS','Total Expenses','Marketing','Budget Sales','Budget Profit','Budget COGS','Budget Margin']].describe()


,Area Code,Inventory,ProductId,Sales,COGS,Total Expenses,Marketing,Budget Sales,Budget Profit,Budget COGS,Budget Margin
count,4248.000000,4248.000000,4248.000000,4248.000000,4248.000000,4248.000000,4248.000000,4248.000000,4248.000000,4248.000000,4248.000000
mean,582.278013,749.381356,6.887006,192.987524,84.433145,54.063559,31.185028,175.649718,60.913371,74.830508,100.819209
std,221.140310,661.031896,3.664072,151.133127,67.249769,32.352598,27.023264,148.891522,79.546123,66.238145,92.602725
min,203.000000,-3534.000000,1.000000,17.000000,0.000000,10.000000,0.000000,0.000000,-320.000000,0.000000,-210.000000
25%,417.000000,432.000000,4.000000,100.000000,43.000000,33.000000,13.000000,80.000000,20.000000,30.000000,50.000000
50%,573.000000,619.000000,6.000000,138.000000,60.000000,46.000000,22.000000,130.000000,40.000000,50.000000,70.000000
75%,772.000000,910.500000,10.000000,230.000000,100.000000,65.000000,39.000000,210.000000,80.000000,90.000000,130.000000
max,985.000000,8252.000000,13.000000,912.000000,364.000000,190.000000,156.000000,1140.000000,560.000000,450.000000,690.000000


- Langkah selanjutnya adalah pengecekan angka negatif diluar dari column yang bersifat numerik atau yang bersifat perhitungan untuk menjaga konsistensi data.

- **Inventory Column** menunjukan minimum value data angka negatif (-3534.000000), hal tersebut seharusnya tidak mungkin terjadi karena Inventaris negatif berarti ada lebih banyak produk yang dijual daripada yang tersedia secara fisik. Saat stok habis, nilai inventaris seharusnya menjadi nol, bukan menjadi negatif. Untuk menghindari analisa dan pengambilan keputusan yang salah, maka selanjutnya transformasi akan dilakukan dengan mengubah bentuk angka negatif menjadi = 0 menggunakan library Pyspark.

In [7]:
# Computing data structure info
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4248 entries, 0 to 4247
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Area Code       4248 non-null   int64  
 1   State           4248 non-null   object 
 2   Market          4248 non-null   object 
 3   Market Size     4248 non-null   object 
 4   Profit          4248 non-null   float64
 5   Margin          4248 non-null   float64
 6   Sales           4248 non-null   float64
 7   COGS            4248 non-null   float64
 8   Total Expenses  4248 non-null   float64
 9   Marketing       4248 non-null   float64
 10  Inventory       4248 non-null   float64
 11  Budget Profit   4248 non-null   float64
 12  Budget COGS     4248 non-null   float64
 13  Budget Margin   4248 non-null   float64
 14  Budget Sales    4248 non-null   float64
 15  ProductId       4248 non-null   int64  
 16  Date            4248 non-null   object 
 17  Product Type    4248 non-null   o

- Selanjutnya adalah mengecek tipe data

- **Date Column** menunjukan tipe data object, hal ini akan menimbulkan kesulitan saat proses pengolahan dan penelitian karena adanya ketidak konsistenan data, contohnya apabila melakukan proses load data ke PostgresSQL, sehingga perlu dilakukan perubahan dari bentuk tipe data object menjadi tipe data date dengan menggunakan library Pyspark pada tahap transformasi.

In [8]:
# Computing nunique function
df[['Area Code','State','Market','ProductId', 'Product Type','Product','Type']].nunique()

Area Code       156
State            20
Market            4
ProductId        13
Product Type      4
Product          13
Type              2
dtype: int64

- Kemudian mengecek kolom categorycal uniqe value pada dataset

- **Column Area Code** memiliki uniqe number yang terlalu banyak yaitu total 156, yang akan menyulitkan dalam proses penelitian lanjutan seperti mencari kolerasi dan prediksi terutama jika data diubah menjadi format one-hot encoding. Selain itu sudah terdapat kolom State untuk menggambarkan area sehingga kolom Area Code akan dihapus pada proses transformasi dengan library Pyspark.

- **Column Product** memiliki uniqe value dan fungsi yang kurang lebih sama dengan kolom ProductId, sehingga ekstistensi nya tidak diperlukan lagi karena sudah diwakilkan keberadaannya dengan ProductId, sehingga column Product akan dihapus pada proses transformasi dengan library Pyspark agar dataset lebih relevan dan efektif.

In [16]:
#show column top 3 
df[['Profit','Margin','Sales','Budget Profit','Budget Margin']].head(3)

,Profit,Margin,Sales,Budget Profit,Budget Margin
0,107.0,176.0,292.0,110.0,160.0
1,75.0,135.0,225.0,90.0,130.0
2,122.0,195.0,325.0,130.0,180.0


- Langkah selanjutnya pengecekan data redundant

-  **Column Profit** dan **Column Margin** merupakan turunan langsung dari column Sales, sama halnya dengan cost dan revenue, tidak menambah informasi baru tetapi hanya mencerminkan hubungan matematis antara fitur lainnya. Redundansi ini bisa membingungkan dalam analisis dan membuat dataset tidak efisien. Selanjutnya akan dihapus pada proses transformasi data menggunakan library Pyspark.

In [10]:
#show column with object data type
df[['State','Market Size','Market','Type','Product Type']].head(3)

,State,Market Size,Market,Type,Product Type
0,Connecticut,Small Market,East,Regular,Coffee
1,Connecticut,Small Market,East,Regular,Coffee
2,Connecticut,Small Market,East,Regular,Coffee


- Langkah selanjutnya adalah merubah huruf pada setiap baris pada kolom yang bertipe data object yang awalnya diawali dengan huruf kapital menjadi menjadi small letter, dengan tujuan menjaga konstistensi dan kebersihan data. 

In [11]:
#show columns
df.columns

Index(['Area Code', 'State', 'Market', 'Market Size', 'Profit', 'Margin',
       'Sales', 'COGS', 'Total Expenses', 'Marketing', 'Inventory',
       'Budget Profit', 'Budget COGS', 'Budget Margin', 'Budget Sales',
       'ProductId', 'Date', 'Product Type', 'Product', 'Type'],
      dtype='object')

- Langkah terakhir agar menghasilkan data yang bersih dan tidak menimbulkan error (contohnya apabila di load ke PostgresSQL) adalah mengubah seluruh nama kolom dengan gabungan huruf kapital menjadi small letter dan mengganti spasi yang awalnya sebagai pemisah antar kata menjadi under score. Selain itu lebih umum digunakan oleh programmer juga kompatibel dengan berbagai alat dan bahasa pemrograman seperti Python, SQL, dan Spark translate





## Summary of Data Exploration

- Here are the facts, insights, and steps that need to be taken as a result of exploring us-store-sales dataset :

- **Data Consistency and Stability:**

- Object-type columns were standardized by converting all text to lowercase to ensure consistency, data cleanliness, and prevent errors in queries or joins caused by case sensitivity.

- Column names were formatted in lowercase with underscores to improve consistency, compatibility, and ease of management, In addition, it is more commonly used by programmers and is also compatible with various tools and programming languages such as Python, SQL, and Spark.

- **Column Redundancy:**

- Profit and Margin columns, derived directly from Sales, do not provide new information and only reflect mathematical relationships. These will be removed during transformation using PySpark to simplify the dataset.

- **High Cardinality Issue:**

- The Area Code column contains too many unique values, which complicates advanced analysis (e.g., correlation or prediction). Its role is already represented by the State column, so it will be dropped during transformation.

- **Duplicate Information:**

- The Product column is redundant as its role is covered by ProductId. It will also be removed during the transformation process to enhance dataset efficiency.

- **Invalid Inventory Values:**

- The Inventory column includes negative values (e.g., -3534), which are unrealistic since inventory cannot be negative. Negative values will be corrected to 0 during transformation to avoid inaccurate analysis.

- **Date Format Inconsistency:**

- The Date column is stored as an object type, which may cause issues during processing, especially in databases like PostgreSQL. It will be converted to a date type using PySpark during the transformation phase for consistency and usability.


- This preparation ensures the dataset is clean, efficient, and ready for further analysis or machine learning tasks.








- Google Slide Presentation : https://docs.google.com/presentation/d/1tWFU8cy0lXD5c1AtrZStEPrXmw9MVlt0eueMK0YqbL0/edit#slide=id.p

## PySpark Data Cleaning and Transformation Procces

In [12]:
# configure and create a new SparkSession instance.
spark = SparkSession.builder.getOrCreate()
spark

FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/spark/./bin/spark-submit'

- Membuat atau Mengakses SparkSession: Jika belum ada sesi Spark yang aktif, fungsi ini akan membuat sesi Spark baru, dan apabila sudah ada sesi aktif, fungsi ini akan mengembalikan sesi yang sudah ada dengan tujuan memastikan hanya satu instance SparkSession yang aktif dalam satu aplikasi.

In [16]:
#Loading dataset to PySpark
sales = spark.read.csv('/Users/mac/Documents/Hacktiv8/M3/p2-coda001-rmt-m3-mirzasjarief/us_store_sales.csv', header=True, inferSchema=True)
sales.show()

+---------+-----------+------+------------+------+------+-----+-----+--------------+---------+---------+-------------+-----------+-------------+------------+---------+-----------------+------------+-----------+-------+
|Area Code|      State|Market| Market Size|Profit|Margin|Sales| COGS|Total Expenses|Marketing|Inventory|Budget Profit|Budget COGS|Budget Margin|Budget Sales|ProductId|             Date|Product Type|    Product|   Type|
+---------+-----------+------+------------+------+------+-----+-----+--------------+---------+---------+-------------+-----------+-------------+------------+---------+-----------------+------------+-----------+-------+
|      203|Connecticut|  East|Small Market| 107.0| 176.0|292.0|116.0|          69.0|     38.0|    962.0|        110.0|      110.0|        160.0|       270.0|        2|04/01/10 00:00:00|      Coffee|  Columbian|Regular|
|      203|Connecticut|  East|Small Market|  75.0| 135.0|225.0| 90.0|          60.0|     29.0|   1148.0|         90.0|      

- Selanjutnya adalah load dataset menggunakan  PySpark untuk proses transformasi data

### Proses transform 1 

- Mengubah angka negatif pada Column Inventory menjadi 0

In [17]:
#Compute code for replacing negative value with 0 in pyspark
sales = sales.withColumn('Inventory', when(col('Inventory') < 0 , 0 ).otherwise(col('Inventory')))

- Code mengecek apakah nilai dalam kolom Inventory lebih kecil dari 0.
Jika ya, maka nilai tersebut akan diganti dengan 0. Namun jika tidak memenuhi kondisi, maka akan tetap menggunakan nilai asli dari kolom Inventory.

In [18]:
#Check minimum value of inventory
sales[['Inventory']].describe().show()

+-------+-----------------+
|summary|        Inventory|
+-------+-----------------+
|  count|             4248|
|   mean|769.1153483992467|
| stddev|610.0801048043983|
|    min|              0.0|
|    max|           8252.0|
+-------+-----------------+



- Sekali lagi memastikan minimun data dari inventory sudah 0, karena logikanya saat stok habis nilai inventaris seharusnya menjadi nol, bukan menjadi negatif, ini bertujuan untuk menghindari analisa dan pengambilan keputusan yang salah pada penelitian lanjutan.

### Proses transform 2

- Mengubah tipe data pada **Column Date**, dan **Inventory**


In [19]:
#Computing convert object to timestamp
sales = sales.withColumn('Date', to_timestamp('Date','dd/MM/yy HH:mm:ss'))

sales[['Date']].show(5)


+-------------------+
|               Date|
+-------------------+
|2010-01-04 00:00:00|
|2010-01-07 00:00:00|
|2010-01-11 00:00:00|
|2010-01-12 00:00:00|
|2011-01-07 00:00:00|
+-------------------+
only showing top 5 rows



- Pertama adalah merubah tipe data object pada column date menjadi tipe data time stamp, dengan tujuan menyamakan bentuk index karena apabila langsung di convert ke dalam bentuk date maka code akan error.

In [20]:
#Computing convert timestamp to date

sales = sales.withColumn("Date", to_date("Date", "yyyy-MM-dd"))

sales[['Date']].show(5)


+----------+
|      Date|
+----------+
|2010-01-04|
|2010-01-07|
|2010-01-11|
|2010-01-12|
|2011-01-07|
+----------+
only showing top 5 rows



- Karena pada keterangan jam menit dan detik kosong (00:00:00), maka agar tidak menyebabkan hal yang ambigu dan menyebabkan error, seperti contohnya ketika proses load ke PostgresSQL maka tipe data timestamps akan diubah kedalam bentuk date / tanggal saja. 

### Proses transform 3



In [21]:
#Computing column drop
sales = sales.drop('Area Code','Product','Margin','Profit')

sales.show(3)


+-----------+------+------------+-----+-----+--------------+---------+---------+-------------+-----------+-------------+------------+---------+----------+------------+-------+
|      State|Market| Market Size|Sales| COGS|Total Expenses|Marketing|Inventory|Budget Profit|Budget COGS|Budget Margin|Budget Sales|ProductId|      Date|Product Type|   Type|
+-----------+------+------------+-----+-----+--------------+---------+---------+-------------+-----------+-------------+------------+---------+----------+------------+-------+
|Connecticut|  East|Small Market|292.0|116.0|          69.0|     38.0|    962.0|        110.0|      110.0|        160.0|       270.0|        2|2010-01-04|      Coffee|Regular|
|Connecticut|  East|Small Market|225.0| 90.0|          60.0|     29.0|   1148.0|         90.0|       80.0|        130.0|       210.0|        2|2010-01-07|      Coffee|Regular|
|Connecticut|  East|Small Market|325.0|130.0|          73.0|     42.0|   1134.0|        130.0|      110.0|        180.0|



- Drop Column Area Code, Product, Margin, Profit


### Proses transform 4



In [22]:
#Convert capital to lower case
sales = (
    sales
    .withColumn('State', lower(col('State')))
    .withColumn('Market', lower(col('Market')))
    .withColumn('Market Size', lower(col('Market Size')))
    .withColumn('Product Type', lower(col('Product Type')))
    .withColumn('Type', lower(col('Type'))))

sales[['State','Market','Market Size','Product Type','Type']].show(5)

+-----------+------+------------+------------+-------+
|      State|Market| Market Size|Product Type|   Type|
+-----------+------+------------+------------+-------+
|connecticut|  east|small market|      coffee|regular|
|connecticut|  east|small market|      coffee|regular|
|connecticut|  east|small market|      coffee|regular|
|connecticut|  east|small market|      coffee|regular|
|connecticut|  east|small market|      coffee|regular|
+-----------+------+------------+------------+-------+
only showing top 5 rows



-  Mengubah huruf kalimat pada index tiap kolom yang bertipe object, memastikan agar data lebih konsisten


### Proses transform 5


In [23]:
#Converting column title from mix to lower case
sales = (
    sales
    .withColumnRenamed("State", "state")
    .withColumnRenamed("Market", "market")
    .withColumnRenamed("Market Size", "market_size")
    .withColumnRenamed("Sales", "sales")
    .withColumnRenamed("COGS", "cogs")
    .withColumnRenamed("Total Expenses", "total_expenses")
    .withColumnRenamed("Marketing", "marketing")
    .withColumnRenamed("Inventory", "inventory")
    .withColumnRenamed("Budget Profit", "budget_profit")
    .withColumnRenamed("Budget COGS", "budget_cogs")
    .withColumnRenamed("Budget Margin", "budget_margin")
    .withColumnRenamed("Budget Sales", "budget_sales")
    .withColumnRenamed("ProductId", "product_id")
    .withColumnRenamed("Date", "date")
    .withColumnRenamed("Product Type", "product_type")
    .withColumnRenamed("Type", "type"))


sales.show(5)

+-----------+------+------------+-----+-----+--------------+---------+---------+-------------+-----------+-------------+------------+----------+----------+------------+-------+
|      state|market| market_size|sales| cogs|total_expenses|marketing|inventory|budget_profit|budget_cogs|budget_margin|budget_sales|product_id|      date|product_type|   type|
+-----------+------+------------+-----+-----+--------------+---------+---------+-------------+-----------+-------------+------------+----------+----------+------------+-------+
|connecticut|  east|small market|292.0|116.0|          69.0|     38.0|    962.0|        110.0|      110.0|        160.0|       270.0|         2|2010-01-04|      coffee|regular|
|connecticut|  east|small market|225.0| 90.0|          60.0|     29.0|   1148.0|         90.0|       80.0|        130.0|       210.0|         2|2010-01-07|      coffee|regular|
|connecticut|  east|small market|325.0|130.0|          73.0|     42.0|   1134.0|        130.0|      110.0|        1

- Fungsi merubah huruf kapital menjadi small letter dan mengganti spasi dengan underscore pada nama column

### Proses transform 6
- Menggabungkan beberapa kolom untuk membuat table baru yang berisi unique value



In [24]:
#Computing new unique value column
sales = sales.withColumn (
    'purchased_id', abs(hash(concat_ws('',col('sales').cast('string'),col('cogs').cast('string'),col('inventory').cast('string'),monotonically_increasing_id().cast('string') )))
    )

sales[['purchased_id']].show(5)

+------------+
|purchased_id|
+------------+
|   491458380|
|  1738729669|
|  1041025097|
|   366943985|
|  1130897045|
+------------+
only showing top 5 rows



- Terakhir adalah proses menambahkan column baru yang merupakan gabungan antara sales.cogs, dan inventory menjadi kode unik untuk menandakan setiap transaksi penjualan dengan nama column **purchased_id**, dengan tujuan untuk memenuhi kriteria validasi data yang memerlukan column dengan unique value.